# Import module and data
---

In [1]:
#########################################################
# Imports modules
#########################################################

import os
import array
import argparse
import base64
import csv
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from ast import literal_eval
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
my_path = os.getcwd()

---
# 1. Label Table
### (1) x_data : decoded waveform

In [5]:
## You have to set dir where decoded data exist
x_folder =  '/home/ubuntu/dr-you-ecg-20220420_mount/220425_SevMUSE_EKG_waveform'
x_data = os.listdir(x_folder)
x_data.sort()

x_data[0], x_data[-1]

('2021100522052936000_2020_3040838.0.csv',
 '2021102622420146000_2007_3921587.csv')

In [6]:
len(x_data)

2647074

In [8]:
# check the 'x_data' shape as dataframe with an example
x_data_0 = pd.read_csv('/home/ubuntu/dr-you-ecg-20220420_mount/220425_SevMUSE_EKG_waveform/2021100522052936000_2020_3040838.0.csv')
print(x_data_0) 

        I   II   V1   V2   V3   V4   V5   V6  III  aVR  aVL  aVF
0     263  195  -58  -97  -97  302  527  458  -68 -229  165   63
1     263  195  -58  -97  -97  302  527  458  -68 -229  165   63
2     263  195  -58  -97  -97  302  527  458  -68 -229  165   63
3     263  195  -58  -97  -97  302  527  458  -68 -229  165   63
4     170   82    0 -131 -248   78  283  204  -88 -126  129   -3
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
4995   29 -492  351   63  185  341  409  375 -521  231  275 -506
4996    9 -507  361   68  185  341  409  370 -516  249  262 -511
4997   -9 -522  370   73  185  341  409  366 -513  265  252 -517
4998  -29 -536  380   78  185  341  409  361 -507  282  239 -521
4999  -43 -536  380   78  185  331  405  351 -493  289  225 -514

[5000 rows x 12 columns]


---
### (2) table including y_data : komate_label_table.csv 

In [10]:
komate_label = pd.read_csv(my_path+'/KOMATE_train_label_220426.csv')
komate_label

,ecg_id,LESION_DATE,AlsUnitNo,filename,STEMI_1
0,2021102313292619000,2006-12-28,3477509.0,2021102313292619000_2006_3477509.csv,0
1,2021102314204087000,2006-12-14,2955848.0,2021102314204087000_2006_2955848.csv,0
2,2021101909220816000,2013-10-06,2314320.0,2021101909220816000_2013_2314320.csv,1
3,2021102314291301000,2006-12-11,2864602.0,2021102314291301000_2006_2864602.csv,0
4,2021102314185556000,2006-12-14,2725918.0,2021102314185556000_2006_2725918.csv,0
...,...,...,...,...,...
13010,2021100612002231000,2020-10-14,9714387.0,2021100612002231000_2020_9714387.csv,0
13011,2021100709053327000,2020-05-20,9762030.0,2021100709053327000_2020_9762030.csv,0
13012,2021100807473531000,2020-01-02,9813451.0,2021100807473531000_2020_9813451.csv,0
13013,2021100711194578000,2020-05-05,9813241.0,2021100711194578000_2020_9813241.csv,0


---

### ※ make  a new filename of 'x_data'
- Now, you cannot use 'x_data' to merge with 'komate_label table' on filename because the name of each x_data(decoded waveform) include '.0' before '.csv' unlike the filename of komate_label table.
- So you have to make a new 'label' to be replaced with the name of each x_data.

In [7]:
# check the whole elements of 'x_data'
x_data

['2021100522052936000_2020_3040838.0.csv',
 '2021100522052950000_2020_4294073.0.csv',
 '2021100522052963000_2020_3040838.0.csv',
 '2021100522053040000_2020_9662039.csv',
 '2021100522053074000_2020_9662039.csv',
 '2021100522053087000_2020_9662039.csv',
 '2021100522053149000_2020_3750147.csv',
 '2021100522053161000_2020_3223148.csv',
 '2021100522053175000_2020_3738302.csv',
 '2021100522053221000_2020_9652850.csv',
 '2021100522053327000_2020_3716767.0.csv',
 '2021100522053425000_2020_3037017.0.csv',
 '2021100522053439000_2020_9556977.csv',
 '2021100522053452000_2020_4585223.csv',
 '2021100522053553000_2020_1802543.0.csv',
 '2021100522053565000_2020_1802543.0.csv',
 '2021100522053589000_2020_9565318.csv',
 '2021100522053660000_2020_2370047.csv',
 '2021100522053810000_2020_4329972.csv',
 '2021100522053823000_2020_2724105.csv',
 '2021100522053836000_2020_1939361.0.csv',
 '2021100522053914000_2020_2653695.0.csv',
 '2021100522053960000_2020_4970750.0.csv',
 '2021100522053986000_2020_4775573.cs

In [24]:
# delete '.0' before '.csv' from x_data and save it named as 'label'
df_fname = pd.DataFrame(x_data)
df_fname = df_fname.rename(columns = {0:'waveform'})
df_fname['label'] = 0

fname_list = []
for i in range(len(df_fname)):
    data = str(df_fname['waveform'][i]).split(".")[0]
    fname = data+ ".csv"
    fname_list.append(fname)

df_fname['label'] = fname_list

df_fname

,waveform,label
0,2021100522052936000_2020_3040838.0.csv,2021100522052936000_2020_3040838.csv
1,2021100522052950000_2020_4294073.0.csv,2021100522052950000_2020_4294073.csv
2,2021100522052963000_2020_3040838.0.csv,2021100522052963000_2020_3040838.csv
3,2021100522053040000_2020_9662039.csv,2021100522053040000_2020_9662039.csv
4,2021100522053074000_2020_9662039.csv,2021100522053074000_2020_9662039.csv
...,...,...
2647069,2021102622414215000_2007_3546246.csv,2021102622414215000_2007_3546246.csv
2647070,2021102622414907000_2007_3374985.csv,2021102622414907000_2007_3374985.csv
2647071,2021102622415031000_2007_3334553.csv,2021102622415031000_2007_3334553.csv
2647072,2021102622415393000_2007_3466103.csv,2021102622415393000_2007_3466103.csv


---
### (3) import the waveform(one part of x_data) of komate_label table

In [27]:
fname_match = pd.merge(komate_label, df_fname, left_on = 'filename', right_on='label', how='inner')
fname_match = fname_match.drop(['filename', 'label'], axis=1)
fname_match.rename(columns = {'waveform':'filename'})
fname_match

,ecg_id,LESION_DATE,AlsUnitNo,STEMI_1,waveform
0,2021102313292619000,2006-12-28,3477509.0,0,2021102313292619000_2006_3477509.csv
1,2021102314204087000,2006-12-14,2955848.0,0,2021102314204087000_2006_2955848.csv
2,2021101909220816000,2013-10-06,2314320.0,1,2021101909220816000_2013_2314320.csv
3,2021102314291301000,2006-12-11,2864602.0,0,2021102314291301000_2006_2864602.csv
4,2021102314185556000,2006-12-14,2725918.0,0,2021102314185556000_2006_2725918.csv
...,...,...,...,...,...
13010,2021100612002231000,2020-10-14,9714387.0,0,2021100612002231000_2020_9714387.csv
13011,2021100709053327000,2020-05-20,9762030.0,0,2021100709053327000_2020_9762030.csv
13012,2021100807473531000,2020-01-02,9813451.0,0,2021100807473531000_2020_9813451.csv
13013,2021100711194578000,2020-05-05,9813241.0,0,2021100711194578000_2020_9813241.csv


In [28]:
fname_match.shape

(13015, 5)

In [33]:
fname_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13015 entries, 0 to 13014
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ecg_id       13015 non-null  int64  
 1   LESION_DATE  13015 non-null  object 
 2   AlsUnitNo    13015 non-null  float64
 3   STEMI_1      13015 non-null  int64  
 4   waveform     13015 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 610.1+ KB


---
# 2. Dataset as input of DL Model

### (1) filename(x_dataset), label(y_dataset) → list → numpy

In [36]:
# Series → list
x_list = fname_match['waveform'].values.tolist()
y_list = fname_match['STEMI_1'].values.tolist()

In [39]:
x_list

['2021102313292619000_2006_3477509.csv',
 '2021102314204087000_2006_2955848.csv',
 '2021101909220816000_2013_2314320.csv',
 '2021102314291301000_2006_2864602.csv',
 '2021102314185556000_2006_2725918.csv',
 '2021102314075153000_2006_2119407.csv',
 '2021102313400946000_2006_3439197.csv',
 '2021102314201241000_2006_4704184.csv',
 '2021102314162784000_2006_2101373.csv',
 '2021102314013546000_2006_2931736.csv',
 '2021102313423116000_2006_2229035.csv',
 '2021102313250757000_2006_2177459.csv',
 '2021102313330035000_2006_2970603.csv',
 '2021102313253442000_2006_1925210.csv',
 '2021102313191853000_2006_3546246.csv',
 '2021102313232305000_2006_3834846.csv',
 '2021102313232480000_2006_4789781.csv',
 '2021102313281371000_2006_4250116.csv',
 '2021102313325931000_2006_2383970.csv',
 '2021102313342300000_2006_2213132.csv',
 '2021102313344978000_2006_2396094.csv',
 '2021102313350802000_2006_4799559.csv',
 '2021102313402758000_2006_2236610.csv',
 '2021102313475572000_2006_2337931.csv',
 '20211023134837

In [37]:
# List → numpy array
x_npy = np.array(x_list)
y_npy = np.array(y_list)

In [38]:
x_npy.shape, y_npy.shape

((13015,), (13015,))

---
### (2) x_dataset
- zero padded to 5120
- ECG Lead 12 to 8

In [42]:
x_ecg = []
x_dataset = []
numList = []

for i in tqdm(x_list):
    x_dir = x_folder +'/'+ i
    df_x = pd.read_csv(x_dir)
    if len(df_x) == 5000 :
        df_x_lead = df_x[['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']]
        a = x_list.index(i)
        numList.append(a)
        df_x_numpy = df_x_lead.to_numpy()
        padding = np.pad(df_x_numpy, ((120,0),(0,0)),'constant',constant_values=0) # Lead zero padded to 5120
        x_dataset.append(padding)

100%|█████████████████████████████████████| 13015/13015 [20:57<00:00, 10.35it/s]


In [102]:
x_train_dataset = np.array(x_dataset)
x_train_dataset.shape

(11394, 5120, 8)

In [106]:
# save
np.save(my_path+'/x_dataset_0426',x_train_dataset)

---
### (3) y_dataset

In [111]:
y_list_final = []

for i in numList :
    y_list_final.append(y_list[i])
    
y_npy = np.array(y_list_final)
y_train_dataset = y_npy
y_train_dataset.shape

(11394,)

In [112]:
# save
np.save(my_path+'/y_dataset_0426',y_dataset_final)

---
### (4) edit the 'komate_label table' by deleting data where waveform is not 5000 rows

In [113]:
komate_label_final = fname_match.loc[numList]

In [115]:
komate_label_final.reset_index(drop=True, inplace=True)
komate_label_final

,ecg_id,LESION_DATE,AlsUnitNo,STEMI_1,waveform
0,2021102314204087000,2006-12-14,2955848.0,0,2021102314204087000_2006_2955848.csv
1,2021101909220816000,2013-10-06,2314320.0,1,2021101909220816000_2013_2314320.csv
2,2021102314201241000,2006-12-14,4704184.0,1,2021102314201241000_2006_4704184.csv
3,2021102314162784000,2006-12-15,2101373.0,0,2021102314162784000_2006_2101373.csv
4,2021102313191853000,2006-12-31,3546246.0,0,2021102313191853000_2006_3546246.csv
...,...,...,...,...,...
11389,2021100612002231000,2020-10-14,9714387.0,0,2021100612002231000_2020_9714387.csv
11390,2021100709053327000,2020-05-20,9762030.0,0,2021100709053327000_2020_9762030.csv
11391,2021100807473531000,2020-01-02,9813451.0,0,2021100807473531000_2020_9813451.csv
11392,2021100711194578000,2020-05-05,9813241.0,0,2021100711194578000_2020_9813241.csv


In [104]:
# final label table
komate_label_final.to_csv(my_path +"/KOMATE_FINAL_train_label_220426.csv")